# Case Study - Balanced Tree Clothing Co.

## High Level Sales Analysis

---

### 1. What was the total quantity sold for all products?

    SELECT product_name, SUM(qty)
    FROM balanced_tree.sales AS s
    JOIN balanced_tree.product_details AS d
    ON s.prod_id = d.product_id
    GROUP BY product_name;

| product_name                     | sum  |
| -------------------------------- | ---- |
| White Tee Shirt - Mens           | 3800 |
| Navy Solid Socks - Mens          | 3792 |
| Grey Fashion Jacket - Womens     | 3876 |
| Navy Oversized Jeans - Womens    | 3856 |
| Pink Fluro Polkadot Socks - Mens | 3770 |
| Khaki Suit Jacket - Womens       | 3752 |
| Black Straight Jeans - Womens    | 3786 |
| White Striped Socks - Mens       | 3655 |
| Blue Polo Shirt - Mens           | 3819 |
| Indigo Rain Jacket - Womens      | 3757 |
| Cream Relaxed Jeans - Womens     | 3707 |
| Teal Button Up Shirt - Mens      | 3646 |

---

---

### 2. What is the total generated revenue for all products before discounts?

    SELECT product_name, SUM(s.qty*s.price) AS "total"
    FROM balanced_tree.sales AS s
    JOIN balanced_tree.product_details AS d
    ON s.prod_id = d.product_id
    GROUP BY product_name;

| product_name                     | total  |
| -------------------------------- | ------ |
| White Tee Shirt - Mens           | 152000 |
| Navy Solid Socks - Mens          | 136512 |
| Grey Fashion Jacket - Womens     | 209304 |
| Navy Oversized Jeans - Womens    | 50128  |
| Pink Fluro Polkadot Socks - Mens | 109330 |
| Khaki Suit Jacket - Womens       | 86296  |
| Black Straight Jeans - Womens    | 121152 |
| White Striped Socks - Mens       | 62135  |
| Blue Polo Shirt - Mens           | 217683 |
| Indigo Rain Jacket - Womens      | 71383  |
| Cream Relaxed Jeans - Womens     | 37070  |
| Teal Button Up Shirt - Mens      | 36460  |

---

---

### 3. What was the total discount amount for all products?

    SELECT product_name, ROUND(SUM(((s.discount/100.00) * s.price)*s.qty),2) AS "total discount"
    FROM balanced_tree.sales AS s
    JOIN balanced_tree.product_details AS d
    ON s.prod_id = d.product_id
    GROUP BY product_name;

| product_name                     | total discount |
| -------------------------------- | -------------- |
| White Tee Shirt - Mens           | 18377.60       |
| Navy Solid Socks - Mens          | 16650.36       |
| Grey Fashion Jacket - Womens     | 25391.88       |
| Navy Oversized Jeans - Womens    | 6135.61        |
| Pink Fluro Polkadot Socks - Mens | 12952.27       |
| Khaki Suit Jacket - Womens       | 10243.05       |
| Black Straight Jeans - Womens    | 14744.96       |
| White Striped Socks - Mens       | 7410.81        |
| Blue Polo Shirt - Mens           | 26819.07       |
| Indigo Rain Jacket - Womens      | 8642.53        |
| Cream Relaxed Jeans - Womens     | 4463.40        |
| Teal Button Up Shirt - Mens      | 4397.60        |

---

## Transaction Analysis

---

### 1. How many unique transactions were there?

    SELECT COUNT (DISTINCT txn_id) AS "Unique Transactions"
    FROM balanced_tree.sales;

| Unique Transactions |
| ------------------- |
| 2500                |

---

---

### 2. What is the average unique products purchased in each transaction?

    SELECT CAST(AVG(unique_products) AS int) AS "avg unique products"
    FROM (
    	SELECT txn_id, COUNT (DISTINCT prod_id) AS "unique_products"
    	FROM balanced_tree.sales
    	GROUP BY txn_id
      ) AS sub_query;

| avg unique products |
| ------------------- |
| 6                   |

---

---

### 3. What are the 25th, 50th and 75th percentile values for the revenue per transaction?

    SELECT 
            	PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY revenue) AS p25,
                PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY revenue) AS p50,
                PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY revenue) AS p75
            FROM (
            SELECT txn_id, ROUND(SUM((qty * price) - ((discount * qty)/100.00) * price), 2) AS "revenue"
            FROM balanced_tree.sales
            GROUP BY txn_id
            ) as sub_query;

| p25                | p50     | p75      |
| ------------------ | ------- | -------- |
| 326.40500000000003 | 441.225 | 572.7625 |

---

---

### 4. What is the average discount value per transaction?

    SELECT ROUND(AVG(avg_discount),2) AS "avg_transaction_discount"
    FROM (
    SELECT txn_id, AVG(((discount/100.00) * price)*qty) AS "avg_discount"
    FROM balanced_tree.sales
    GROUP BY txn_id
    ) AS sub_query;

| avg_transaction_discount |
| ------------------------ |
| 10.28                    |

---

---

### 5. What is the percentage split of all transactions for members vs non-members?

    SELECT 
    	member, 
        (CAST(COUNT(member) AS FLOAT)/(SELECT COUNT(*) FROM balanced_tree.sales)) * 100 as member_percentages
    FROM balanced_tree.sales
    GROUP BY member;

| member | member_percentages                |
| ------ | ----------------- |
| false  | 39.97350115932428 |
| true   | 60.02649884067572 |

---

---

### 6. What is the average revenue for member transactions and non-member transactions?

    WITH transactions AS (
    SELECT 
        member,
        txn_id,
        ROUND(SUM((qty * price) - ((discount * qty)/100.00) * price), 2) AS "revenue"
    FROM balanced_tree.sales
    GROUP BY member, txn_id
    )
    
    SELECT member, AVG(revenue) AS avg_revenue
    FROM transactions
    GROUP BY member;

| member | avg_revenue          |
| ------ | -------------------- |
| false  | 452.0077688442211055 |
| true   | 454.1369634551495017 |

---

## Product Analysis

---

### 1. What are the top 3 products by total revenue before discount?

    SELECT 
    	product_name,
    	ROUND(SUM(s.qty * s.price), 2) AS "revenue"
    FROM balanced_tree.sales AS s
    JOIN balanced_tree.product_details AS d
    ON s.prod_id = d.product_id
    GROUP BY product_name
    ORDER BY revenue DESC
    LIMIT 3;

| product_name                 | revenue   |
| ---------------------------- | --------- |
| Blue Polo Shirt - Mens       | 217683.00 |
| Grey Fashion Jacket - Womens | 209304.00 |
| White Tee Shirt - Mens       | 152000.00 |

---

---

### 2. What is the total quantity, revenue and discount for each segment?

    SELECT segment_name, SUM(s.qty) AS total_quantity, SUM(s.qty*s.price) AS total_revenue, ROUND(SUM(((s.discount/100.00)*s.price)*s.qty),2) AS total_discount
    FROM balanced_tree.sales AS s
    JOIN balanced_tree.product_details AS d
    ON s.prod_id = d.product_id
    GROUP BY segment_name;

| segment_name | total_quantity | total_revenue | total_discount |
| ------------ | -------------- | ------------- | -------------- |
| Shirt        | 11265          | 406143        | 49594.27       |
| Jeans        | 11349          | 208350        | 25343.97       |
| Jacket       | 11385          | 366983        | 44277.46       |
| Socks        | 11217          | 307977        | 37013.44       |

---

---

### 3. What is the top selling product for each segment?

    WITH my_temp AS (
    	SELECT product_name, segment_name, SUM(s.qty) AS sales
    	FROM balanced_tree.sales AS s
    		JOIN balanced_tree.product_details AS d
    		ON s.prod_id = d.product_id
    	GROUP BY product_name, segment_name
    	),
    ranked AS (
    	SELECT product_name, segment_name, sales, ROW_NUMBER() OVER (PARTITION BY segment_name ORDER BY sales DESC) as ranking
    	FROM my_temp
    )
    
    SELECT segment_name, product_name, sales
    FROM ranked
    WHERE ranking = 1;

| segment_name | product_name                  | sales |
| ------------ | ----------------------------- | ----- |
| Jacket       | Grey Fashion Jacket - Womens  | 3876  |
| Jeans        | Navy Oversized Jeans - Womens | 3856  |
| Shirt        | Blue Polo Shirt - Mens        | 3819  |
| Socks        | Navy Solid Socks - Mens       | 3792  |

---

---

### 4. What is the total quantity, revenue and discount for each category?

    SELECT category_name, SUM(s.qty) AS total_quantity, SUM(s.qty*s.price) AS total_revenue, ROUND(SUM(((s.discount/100.00)*s.price)*s.qty),2) AS total_discount
    FROM balanced_tree.sales AS s
    JOIN balanced_tree.product_details AS d
    ON s.prod_id = d.product_id
    GROUP BY category_name;

| category_name | total_quantity | total_revenue | total_discount |
| ------------- | -------------- | ------------- | -------------- |
| Mens          | 22482          | 714120        | 86607.71       |
| Womens        | 22734          | 575333        | 69621.43       |

---

---

### 5. What is the top selling product for each category?

    WITH my_temp AS (
    	SELECT category_name, product_name, SUM(s.qty) AS sales
      	FROM balanced_tree.sales AS s
      	JOIN balanced_tree.product_details AS d
      	ON s.prod_id = d.product_id
      	GROUP BY category_name, product_name
    ),
    ranked AS (
    	SELECT category_name, product_name, sales, ROW_NUMBER() OVER (PARTITION BY category_name ORDER BY sales DESC) AS ranking
      	FROM my_temp
    )
    
    SELECT category_name, product_name, sales
    FROM ranked
    WHERE ranking = 1;

| category_name | product_name                 | sales |
| ------------- | ---------------------------- | ----- |
| Mens          | Blue Polo Shirt - Mens       | 3819  |
| Womens        | Grey Fashion Jacket - Womens | 3876  |

---

---

### 6. What is the percentage split of revenue by product for each segment?

    WITH temp_table AS (
        	SELECT segment_name, SUM(s.qty*s.price) AS total
            FROM balanced_tree.sales AS s
            JOIN balanced_tree.product_details AS d
            ON s.prod_id = d.product_id
            GROUP BY segment_name
        )
        
        SELECT d.segment_name, d.product_name, SUM(s.qty*s.price) AS sales, MAX(t.total) AS total, CAST((SUM(s.qty*s.price)/CAST(MAX(t.total) AS FLOAT))*100.00 AS INT) AS percentage_split
        FROM balanced_tree.sales AS s
        JOIN balanced_tree.product_details AS d
        ON s.prod_id = d.product_id
        LEFT JOIN temp_table AS t
        ON d.segment_name = t.segment_name
        GROUP BY d.segment_name, d.product_name
        ORDER BY d.segment_name;

| segment_name | product_name                     | sales  | total  | percentage_split |
| ------------ | -------------------------------- | ------ | ------ | ---------------- |
| Jacket       | Indigo Rain Jacket - Womens      | 71383  | 366983 | 19               |
| Jacket       | Khaki Suit Jacket - Womens       | 86296  | 366983 | 24               |
| Jacket       | Grey Fashion Jacket - Womens     | 209304 | 366983 | 57               |
| Jeans        | Navy Oversized Jeans - Womens    | 50128  | 208350 | 24               |
| Jeans        | Black Straight Jeans - Womens    | 121152 | 208350 | 58               |
| Jeans        | Cream Relaxed Jeans - Womens     | 37070  | 208350 | 18               |
| Shirt        | White Tee Shirt - Mens           | 152000 | 406143 | 37               |
| Shirt        | Blue Polo Shirt - Mens           | 217683 | 406143 | 54               |
| Shirt        | Teal Button Up Shirt - Mens      | 36460  | 406143 | 9                |
| Socks        | Navy Solid Socks - Mens          | 136512 | 307977 | 44               |
| Socks        | White Striped Socks - Mens       | 62135  | 307977 | 20               |
| Socks        | Pink Fluro Polkadot Socks - Mens | 109330 | 307977 | 35               |

---

---

### 7. What is the percentage split of revenue by segment for each category?

    WITH temp_table AS (
        	SELECT category_name, SUM(s.qty*s.price) AS total
            FROM balanced_tree.sales AS s
            JOIN balanced_tree.product_details AS d
            ON s.prod_id = d.product_id
            GROUP BY category_name
        )
        
        SELECT d.category_name, d.segment_name, SUM(s.qty*s.price) AS sales, MAX(t.total) AS total, CAST((SUM(s.qty*s.price)/CAST(MAX(t.total) AS FLOAT))*100.00 AS INT) AS percentage_split
        FROM balanced_tree.sales AS s
        JOIN balanced_tree.product_details AS d
        ON s.prod_id = d.product_id
        LEFT JOIN temp_table AS t
        ON d.category_name = t.category_name
        GROUP BY d.category_name, d.segment_name
        ORDER BY d.category_name;

| category_name | segment_name | sales  | total  | percentage_split |
| ------------- | ------------ | ------ | ------ | ---------------- |
| Mens          | Socks        | 307977 | 714120 | 43               |
| Mens          | Shirt        | 406143 | 714120 | 57               |
| Womens        | Jeans        | 208350 | 575333 | 36               |
| Womens        | Jacket       | 366983 | 575333 | 64               |

---

---

### 8. What is the percentage split of total revenue by category?

    SELECT d.category_name, SUM(s.qty*s.price) AS sales, (SELECT SUM(qty*price) AS total FROM balanced_tree.sales) AS total, CAST((SUM(s.qty*s.price)/CAST((SELECT SUM(qty*price) AS total FROM balanced_tree.sales) AS FLOAT))*100.00 AS INT) AS percentage_split
    FROM balanced_tree.sales AS s
    JOIN balanced_tree.product_details AS d
    ON s.prod_id = d.product_id
    GROUP BY d.category_name
    ORDER BY d.category_name;

| category_name | sales  | total   | percentage_split |
| ------------- | ------ | ------- | ---------------- |
| Mens          | 714120 | 1289453 | 55               |
| Womens        | 575333 | 1289453 | 45               |

---

---

### 9. What is the total transaction “penetration” for each product? (hint: penetration = number of transactions where at least 1 quantity of a product was purchased divided by total number of transactions)


    SELECT 
    	d.product_name, 
        COUNT(DISTINCT s.txn_id) AS product_transactions,
        total_transactions.total AS total_transactions,
        (COUNT(DISTINCT s.txn_id)::FLOAT / total_transactions.total) AS penetration
    FROM balanced_tree.sales AS s
    JOIN balanced_tree.product_details AS d
    ON s.prod_id = d.product_id
    JOIN (
        SELECT COUNT(DISTINCT txn_id) AS total
        FROM balanced_tree.sales
    ) AS total_transactions ON true
    WHERE s.qty > 0
    GROUP BY d.product_name, total_transactions.total
    ORDER BY penetration DESC;

| product_name                     | product_transactions | total_transactions | penetration |
| -------------------------------- | -------------------- | ------------------ | ----------- |
| Navy Solid Socks - Mens          | 1281                 | 2500               | 0.5124      |
| Grey Fashion Jacket - Womens     | 1275                 | 2500               | 0.51        |
| Navy Oversized Jeans - Womens    | 1274                 | 2500               | 0.5096      |
| White Tee Shirt - Mens           | 1268                 | 2500               | 0.5072      |
| Blue Polo Shirt - Mens           | 1268                 | 2500               | 0.5072      |
| Pink Fluro Polkadot Socks - Mens | 1258                 | 2500               | 0.5032      |
| Indigo Rain Jacket - Womens      | 1250                 | 2500               | 0.5         |
| Khaki Suit Jacket - Womens       | 1247                 | 2500               | 0.4988      |
| Black Straight Jeans - Womens    | 1246                 | 2500               | 0.4984      |
| Cream Relaxed Jeans - Womens     | 1243                 | 2500               | 0.4972      |
| White Striped Socks - Mens       | 1243                 | 2500               | 0.4972      |
| Teal Button Up Shirt - Mens      | 1242                 | 2500               | 0.4968      |

---

---

### 10. What is the most common combination of at least 1 quantity of any 3 products in a 1 single transaction?

    WITH transaction_products AS (
        SELECT txn_id, product_name
        FROM balanced_tree.sales AS s
        JOIN balanced_tree.product_details AS d ON s.prod_id = d.product_id
        WHERE s.qty > 0
    ),
    combinations AS (
        SELECT
            tp1.product_name AS product1,
            tp2.product_name AS product2,
            tp3.product_name AS product3,
            tp1.txn_id
        FROM transaction_products AS tp1
        JOIN transaction_products AS tp2 ON tp1.txn_id = tp2.txn_id AND tp1.product_name < tp2.product_name
        JOIN transaction_products AS tp3 ON tp1.txn_id = tp3.txn_id AND tp2.product_name < tp3.product_name
    )
    SELECT
        product1,
        product2,
        product3,
        COUNT(*) AS combination_count
    FROM combinations
    GROUP BY product1, product2, product3
    ORDER BY combination_count DESC
    LIMIT 1;

| product1                     | product2                    | product3               | combination_count |
| ---------------------------- | --------------------------- | ---------------------- | ----------------- |
| Grey Fashion Jacket - Womens | Teal Button Up Shirt - Mens | White Tee Shirt - Mens | 352               |

---